In [8]:
import os 

import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import nbimporter
import Useful_Visualization_Functions
from pyspark.ml import *
from pyspark.sql import *
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import Imputer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import col, explode, array, lit

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

warnings.filterwarnings("ignore")
import pyspark.sql.functions as F

### Build spark session

In [9]:
myspark = SparkSession\
    .builder\
    .appName("AWS-Spark")\
    .config("spark.driver.memory", "200g") \
    .config("spark.sql.shuffle.partitions",6)\
    .config("spark.sql.repl.eagereval.enabled",True)\
    .getOrCreate()

ConnectionRefusedError: [Errno 111] Connection refused

### Import parquet files

In [3]:
df_clean = myspark.read.parquet("clean-noaa")
# noaa_data.show(10)

### Undersampling

In [4]:
zero_df = df_clean.filter(col("ItRained") == 0)
one_df = df_clean.filter(col("ItRained") == 1)

major_df, minor_df = (zero_df, one_df) if zero_df.count() > one_df.count() else (one_df, zero_df)

#if zero_df.count() > one_df.count():
#    major_df = zero_df
#    minor_df = one_df
#else:
#    major_df = one_df
#    minor_df = zero_df
    
ratio = major_df.count()/minor_df.count()
sampled_majority_df = major_df.sample(False, 1/ratio)
df_clean = sampled_majority_df.unionAll(minor_df)
df_clean.select("ItRained").summary().show()

+-------+------------------+
|summary|          ItRained|
+-------+------------------+
|  count|            302520|
|   mean|0.4991108025915642|
| stddev|0.5000000357197292|
|    min|                 0|
|    25%|                 0|
|    50%|                 0|
|    75%|                 1|
|    max|                 1|
+-------+------------------+



In [5]:
cols_interest = ['TEMP','DEWP','SLP','VISIB','WDSP','MXSPD','MAX','MIN']
non_labeled_df = df_clean.select(cols_interest)
labels_col = df_clean.select('TEMP','ItRained')
labels_col.show(10)

+----+--------+
|TEMP|ItRained|
+----+--------+
|12.1|       0|
| 8.0|       0|
|28.1|       0|
|28.8|       0|
|29.4|       0|
|13.4|       0|
|10.0|       0|
|31.0|       0|
|20.8|       0|
|17.1|       0|
+----+--------+
only showing top 10 rows



In [6]:
assembler = VectorAssembler(inputCols=cols_interest,outputCol="features")
non_labeled_df = assembler.transform(non_labeled_df)
# Trains a k-means model.
kmeans = KMeans(featuresCol="features").setK(2).setSeed(1)
model = kmeans.fit(non_labeled_df)

# Make predictions
predictions = model.transform(non_labeled_df)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.6374030922357818
Cluster Centers: 
[  34.05673689   29.0167568  1010.79240146   10.76208255    7.13030535
   11.45582452   39.68107306   27.80810146]
[ 9.86715291e+00  3.36485080e+00  1.02012408e+03  1.40242641e+01
  6.11788847e+00  9.60748213e+00  1.83230696e+01 -4.39274706e-01]


In [7]:
# predictions.orderBy('prediction', ascending=True).show()
predictions_df_sampled = predictions.join(labels_col, "TEMP").sample(False, 0.1).toPandas()
# df_clean = df_clean.select(cols_interest).sample(False, 0.1).toPandas()
# Useful_Visualization_Functions.plotScatter(predictions_df_sampled,"TEMP", "DEWP", "prediction")
Useful_Visualization_Functions.plotScatterMatrix(predictions_df_sampled, "prediction")
Useful_Visualization_Functions.plotScatterMatrix(predictions_df_sampled, "ItRained")

ERROR:root:Exception while sending command.                                     
Traceback (most recent call last):
  File "/home/nuno/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 480, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nuno/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/nuno/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 503, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/nuno/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 480, in send_command
    raise Py4JNetworkE

ValueError: Unable to parse datatype from schema. An error occurred while calling o180.schema